## Build model

In [1]:
import fasttext
import numpy as np

In [2]:
model = fasttext.train_supervised(input="cooking.train", lr=0.8, epoch=70, loss='hs')

Read 0M words
Number of words:  14543
Number of labels: 735
Progress: 100.0% words/sec/thread: 2027877 lr:  0.000000 avg.loss:  4.040552 ETA:   0h 0m 0s


In [3]:
model.test("cooking.valid")

(3000, 0.5, 0.21623180049012541)

In [4]:
class_names = model.labels

In [5]:
class_names = [s.replace("__label__", "") for s in class_names]

In [6]:
k = len(class_names)
idx_to_labels = {i:k for k, i in zip(class_names, range(k))}
labels_to_idx = {k:i for k, i in zip(class_names, range(k))}

In [7]:
labels_to_idx['cake'], k

(10, 735)

In [8]:
def get_predictions(model, text_list):
    
    predictions = model.predict(text_list, k=k)
    x, y = predictions
    
    probabilities_full_list = []
    for label_list, prob_list in zip(x, y):
        label_prob_pair_dict = {}
        for lbl, prob in zip(label_list, prob_list):
            label_prob_pair_dict[lbl.replace("__label__", "")] = prob
        probabilities_list = []
        for cls in class_names:
            if cls in label_prob_pair_dict:
                p = label_prob_pair_dict[cls]
                probabilities_list.append(p)
            else:
                probabilities_list.append(0.0)
        probabilities_full_list.append(probabilities_list)
        
    return np.array(probabilities_full_list)

In [9]:
%%time
results = get_predictions(model, ["cake"]*1000*10)

CPU times: user 1.22 s, sys: 53.5 ms, total: 1.27 s
Wall time: 1.27 s


# Unbox

In [ ]:
import unboxapi
# client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")
unboxapi.DEPLOYMENT = unboxapi.DeploymentType.AWS
unboxapi.api.UNBOX_ENDPOINT = "http://0.0.0.0:8080/api" # Use this for local testing
client = unboxapi.UnboxClient("481bb19f-c58c-415b-a957-36cbfe17ec85")

# add model

In [ ]:
from unboxapi.models import ModelType

model = client.add_model(
    function=get_predictions, 
    model=model,
    model_type=ModelType.fasttext,
    class_names=class_names,
    name='Cooking Fast Text',
    description='this is my fasttext model'
)
model.to_dict()

# add dataset

In [ ]:
small_df.to_csv("cooking_validation.csv", index=False)

In [ ]:
sm_df = pd.read_csv("cooking_validation.csv")

In [ ]:
dataset = client.add_dataframe(
    df=small_df,
    class_names=class_names,
    label_column_name='label',
    text_column_name='text',
    name="Small Cooking Dataset",
    description='this is my fasttext validation dataset'
)
dataset.to_dict()